### Linear Regression - Minimise MAE using Gurobi

In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
data = pd.read_csv('data.csv')
print(data.shape)
data.head()

# Split X & y
X_train = data.drop(columns=['y']).copy()
y_train = data['y']

(50, 4)


In [3]:
# Create a new model
model = gp.Model()

# Coefficient variables
coeffs = model.addVars(X_train.columns, lb=-GRB.INFINITY)

# Intercept
intercept = model.addVar(lb=-GRB.INFINITY)

# Absolute error variables
abs_errors = model.addVars(X_train.index) # Doesn't matter if lb=0 orlb=-GRB.INFINITY

# Objective: Minimize the sum of absolute errors
model.setObjective(gp.quicksum(abs_errors), GRB.MINIMIZE)

# Constraints
for i in X_train.index:
    # Prediction
    prediction = intercept + gp.quicksum(coeffs[j] * X_train.loc[i, j] for j in X_train.columns)
    
    # Constraints to define absolute error
    model.addConstr(prediction - y_train[i] <= abs_errors[i])
    model.addConstr(y_train[i] - prediction <= abs_errors[i])

# Optimize
model.Params.OutputFlag = 0
model.optimize()

# Co-efficients
fitted_coeffs = {j: coeffs[j].X for j in X_train.columns}
fitted_intercept = intercept.X

print("Fitted Coefficients:", fitted_coeffs)
print("Fitted Intercept:", fitted_intercept)
print(f'MAE: {round(model.objval/50,3)}')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-26
Fitted Coefficients: {'x1': -4.111693619345488, 'x2': -0.00851979806807563, 'x3': -4.582930487634735}
Fitted Intercept: 3.9406362651773237
MAE: 0.381


/var/folders/v1/2b2w4wyd2mv05s43hysll80m0000gn/T/ipykernel_2383/3334237888.py:14: DeprecationWarning: Calling quicksum on a tupledict is deprecated, use .sum() instead.
  model.setObjective(gp.quicksum(abs_errors), GRB.MINIMIZE)


In [4]:
# Cross-check using LM from sklearn

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

rm = LinearRegression()
rm.fit(X_train, y_train)
y_pred = rm.predict(X_train)
print(f'MAE: {mean_absolute_error(y_train, y_pred)}')

MAE: 0.3962276944901045
